## **Chapter 5: Data Pre-processing**
### **Ex1: DallasCouncilVoters**
Cho dữ liệu DallasCouncilVoters.csv
Yêu cầu:
1. Đọc dữ liệu => df
2. Cho biết dữ liệu có bao nhiêu dòng, in scheme. Hiển thị 5 dòng dữ liệu đầu tiên.
3. Kiểm tra dữ liệu NaN, null. Nếu dòng nào 'VOTER_NAME' có dữ liệu null thì xóa hết các dòng
đó.
4. Kiểm tra dữ liệu trùng. Xóa dữ liệu trùng.
5. Tìm các VOTER_NAME duy nhất và hiển thị 10 thông tin đầu tiên.
6. Lọc dữ liệu theo điều kiện 'VOTER_NAME' có chiều dài từ 1-20 ký tự.
7. Loại bỏ các dữ liệu mà trong 'VOTER_NAME' có chứa dấu '_' (underscore)
8. Tạo cột 'splits' chứa thông tin được cắt theo khoảng trắng từ 'VOTER_NAME'
9. Tạo cột 'first_name' lấy dữ liệu từ phần tử đầu tiên trong cột 'splits'
10. Tạo cột 'last_name' lấy dữ liệu từ phần tử cuối cùng trong cột 'splits'
11. Tạo cột 'random_val' theo điều kiện: nếu cột 'TITLE' có nội dung là 'Councilmember' thì
'random_val' sẽ có giá trị rand(), nếu có nội dung là 'Mayor' thì 'random_val' sẽ có giá trị là 2,
ngược lại sẽ có giá trị là 0.
12. Lọc các dòng dữ liệu có 'random_val' = 0. Hiển thị.
13. Xây dựng function: getFirstAndMiddle(names) trả về kết quả gồm First và Middle (names).
Khai báo function vừa viết dưới dạng udf đặt tên là udfFirstAndMiddle.
14. Tạo cột first_and_middle_name bằng cách gọi udf trên với tham số truyền vào là cột 'splits'. In
kết quả.
15. Xóa bỏ các cột 'first_name', 'splits'. In kết quả.
16. Thêm cột 'ROW_ID' bằng phương thức: monotonically_increasing_id() (trong
pyspark.sql.functions).
17. Hiển thị 10 dòng đầu của dữ liệu với ROW_ID tăng dần.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.3/spark-3.1.3-bin-hadoop3.2.tgz
!tar xf spark-3.1.3-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.3-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()

In [4]:
# import libraries
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from pyspark.sql.functions import mean, stddev, col, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql import types 
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import coalesce, first, lit
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import datediff
from pyspark.sql.functions import when
from pyspark.sql.functions import *
from pyspark.sql.types import DateType


In [5]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd '/content/gdrive/My Drive/LDS9_K273_ONLINE_Đinh Viết Trung/Chapter_6'

Mounted at /content/gdrive
/content/gdrive/My Drive/LDS9_K273_ONLINE_Đinh Viết Trung/Chapter_6


In [6]:
sc = SparkContext()

In [7]:
spark = SparkSession(sc)

### **1. Đọc dữ liệu => df**

In [101]:
df = spark.read.csv('data/DallasCouncilVoters.csv', header=True, inferSchema=True).cache()

### **2. Cho biết dữ liệu có bao nhiêu dòng, in scheme. Hiển thị 5 dòng dữ liệu đầu tiên**

In [102]:
df.count()

44625

In [103]:
df.printSchema()

root
 |-- DATE: string (nullable = true)
 |-- TITLE: string (nullable = true)
 |-- VOTER_NAME: string (nullable = true)



In [104]:
df.show(5)

+----------+-------------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|
+----------+-------------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|
|02/08/2017|Councilmember|       Casey Thomas|
+----------+-------------+-------------------+
only showing top 5 rows



### **3. Kiểm tra dữ liệu NaN, null. Nếu dòng nào 'VOTER_NAME' có dữ liệu null thì xóa hết các dòng đó**

In [105]:
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).toPandas().T

,0
DATE,0
TITLE,0
VOTER_NAME,0


In [106]:
'''
=> ko có giá trị nan
'''

'\n=> ko có giá trị nan\n'

In [107]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas().T

,0
DATE,0
TITLE,195
VOTER_NAME,503


In [108]:
'''
=> có giá trị null tại Voter_Name => drop đi nào:))
'''

'\n=> có giá trị null tại Voter_Name => drop đi nào:))\n'

In [109]:
df = df.dropna(subset='VOTER_NAME')

In [110]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas().T

,0
DATE,0
TITLE,0
VOTER_NAME,0


### **4. Kiểm tra dữ liệu trùng. Xóa dữ liệu trùng**

In [111]:
total = df.count()
distinct = df.distinct().count()

print(total)
print(distinct)

print(('df có các duplicate records với', total - distinct, 'duplicate records') if total > distinct else 'df không có các duplicate records')

44122
1273
('df có các duplicate records với', 42849, 'duplicate records')


In [112]:
df.filter(df['VOTER_NAME'] == 'Philip T. Kingston').show(5)

+----------+-------------+------------------+
|      DATE|        TITLE|        VOTER_NAME|
+----------+-------------+------------------+
|02/08/2017|Councilmember|Philip T. Kingston|
|02/08/2017|Councilmember|Philip T. Kingston|
|01/11/2017|Councilmember|Philip T. Kingston|
|09/14/2016|Councilmember|Philip T. Kingston|
|01/04/2017|Councilmember|Philip T. Kingston|
+----------+-------------+------------------+
only showing top 5 rows



In [113]:
df = df.drop_duplicates()
df.count()

1273

### **5. Tìm các VOTER_NAME duy nhất và hiển thị 10 thông tin đầu tiên.**

In [114]:
df[['VOTER_NAME']].distinct().show(10)

+--------------------+
|          VOTER_NAME|
+--------------------+
|      Tennell Atkins|
|  the  final   20...|
|        Scott Griggs|
|       Scott  Griggs|
|       Sandy Greyson|
| Michael S. Rawlings|
| the final 2018 A...|
|        Kevin Felder|
|        Adam Medrano|
|       Casey  Thomas|
+--------------------+
only showing top 10 rows



### **6. Lọc dữ liệu theo điều kiện 'VOTER_NAME' có chiều dài từ 1-20 ký tự.**

In [115]:
# cách 1
df_sub1 = df.where((length(col('VOTER_NAME'))>1) & (length(col('VOTER_NAME'))<20))
df_sub1.show()

+----------+--------------------+-------------------+
|      DATE|               TITLE|         VOTER_NAME|
+----------+--------------------+-------------------+
|04/11/2018|Deputy Mayor Pro Tem|       Adam Medrano|
|02/14/2018|       Councilmember|    Lee M. Kleinman|
|04/25/2018|       Councilmember|     Tennell Atkins|
|08/29/2018|       Councilmember|       Kevin Felder|
|10/18/2017|       Councilmember| Jennifer S.  Gates|
|12/13/2017|       Councilmember|     Sandy  Greyson|
|12/13/2017|       Councilmember|       Omar Narvaez|
|01/03/2018|               Mayor|Michael S. Rawlings|
|02/28/2018|       Councilmember|     Sandy  Greyson|
|03/01/2017|       Councilmember|      Sandy Greyson|
|04/12/2017|       Mayor Pro Tem|   Monica R. Alonzo|
|11/09/2016|       Councilmember|  Tiffinni A. Young|
|06/20/2018|       Councilmember|Rickey D.  Callahan|
|02/28/2018|       Councilmember|     Tennell Atkins|
|10/24/2018|       Councilmember|     Sandy  Greyson|
|06/28/2017|Deputy Mayor Pro

In [116]:
# cách 2
df_sub2 = df.filter('length(VOTER_NAME) > 0 and length(VOTER_NAME) < 20')
df_sub2.show()

+----------+--------------------+-------------------+
|      DATE|               TITLE|         VOTER_NAME|
+----------+--------------------+-------------------+
|04/11/2018|Deputy Mayor Pro Tem|       Adam Medrano|
|02/14/2018|       Councilmember|    Lee M. Kleinman|
|04/25/2018|       Councilmember|     Tennell Atkins|
|08/29/2018|       Councilmember|       Kevin Felder|
|10/18/2017|       Councilmember| Jennifer S.  Gates|
|12/13/2017|       Councilmember|     Sandy  Greyson|
|12/13/2017|       Councilmember|       Omar Narvaez|
|01/03/2018|               Mayor|Michael S. Rawlings|
|02/28/2018|       Councilmember|     Sandy  Greyson|
|03/01/2017|       Councilmember|      Sandy Greyson|
|04/12/2017|       Mayor Pro Tem|   Monica R. Alonzo|
|11/09/2016|       Councilmember|  Tiffinni A. Young|
|06/20/2018|       Councilmember|Rickey D.  Callahan|
|02/28/2018|       Councilmember|     Tennell Atkins|
|10/24/2018|       Councilmember|     Sandy  Greyson|
|06/28/2017|Deputy Mayor Pro

### **7. Loại bỏ các dữ liệu mà trong 'VOTER_NAME' có chứa dấu '_' (underscore)**

In [117]:
df_sub3 = df_sub2.filter(~col('VOTER_NAME').contains('_'))

In [118]:
df_sub3.select('VOTER_NAME').distinct().show(10, truncate=False)


+-------------------+
|VOTER_NAME         |
+-------------------+
|Tennell Atkins     |
|Scott Griggs       |
|Scott  Griggs      |
|Sandy Greyson      |
|Michael S. Rawlings|
|Kevin Felder       |
|Adam Medrano       |
|Casey  Thomas      |
|Mark  Clayton      |
|Casey Thomas       |
+-------------------+
only showing top 10 rows



### **8. Tạo cột 'splits' chứa thông tin được cắt theo khoảng trắng từ 'VOTER_NAME'**

In [119]:
df_sub3 = df_sub3.withColumn('splits', split(df.VOTER_NAME, '\s+'))

In [120]:
df_sub3.show(5)

+----------+--------------------+------------------+--------------------+
|      DATE|               TITLE|        VOTER_NAME|              splits|
+----------+--------------------+------------------+--------------------+
|04/11/2018|Deputy Mayor Pro Tem|      Adam Medrano|     [Adam, Medrano]|
|02/14/2018|       Councilmember|   Lee M. Kleinman| [Lee, M., Kleinman]|
|04/25/2018|       Councilmember|    Tennell Atkins|   [Tennell, Atkins]|
|08/29/2018|       Councilmember|      Kevin Felder|     [Kevin, Felder]|
|10/18/2017|       Councilmember|Jennifer S.  Gates|[Jennifer, S., Ga...|
+----------+--------------------+------------------+--------------------+
only showing top 5 rows



### **9. Tạo cột 'first_name' lấy dữ liệu từ phần tử đầu tiên trong cột 'splits'**

In [121]:
df_sub3 = df_sub3.withColumn('first_name', df_sub3.splits.getItem(0))

In [122]:
df_sub3.show(5)

+----------+--------------------+------------------+--------------------+----------+
|      DATE|               TITLE|        VOTER_NAME|              splits|first_name|
+----------+--------------------+------------------+--------------------+----------+
|04/11/2018|Deputy Mayor Pro Tem|      Adam Medrano|     [Adam, Medrano]|      Adam|
|02/14/2018|       Councilmember|   Lee M. Kleinman| [Lee, M., Kleinman]|       Lee|
|04/25/2018|       Councilmember|    Tennell Atkins|   [Tennell, Atkins]|   Tennell|
|08/29/2018|       Councilmember|      Kevin Felder|     [Kevin, Felder]|     Kevin|
|10/18/2017|       Councilmember|Jennifer S.  Gates|[Jennifer, S., Ga...|  Jennifer|
+----------+--------------------+------------------+--------------------+----------+
only showing top 5 rows



### **10. Tạo cột 'last_name' lấy dữ liệu từ phần tử cuối cùng trong cột 'splits**

In [123]:
df_sub3 = df_sub3.withColumn('last_name', df_sub3.splits.getItem(size(df_sub3.splits)-1))

In [124]:
df_sub3.show(5)

+----------+--------------------+------------------+--------------------+----------+---------+
|      DATE|               TITLE|        VOTER_NAME|              splits|first_name|last_name|
+----------+--------------------+------------------+--------------------+----------+---------+
|04/11/2018|Deputy Mayor Pro Tem|      Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|
|02/14/2018|       Councilmember|   Lee M. Kleinman| [Lee, M., Kleinman]|       Lee| Kleinman|
|04/25/2018|       Councilmember|    Tennell Atkins|   [Tennell, Atkins]|   Tennell|   Atkins|
|08/29/2018|       Councilmember|      Kevin Felder|     [Kevin, Felder]|     Kevin|   Felder|
|10/18/2017|       Councilmember|Jennifer S.  Gates|[Jennifer, S., Ga...|  Jennifer|    Gates|
+----------+--------------------+------------------+--------------------+----------+---------+
only showing top 5 rows



### **11. Tạo cột 'random_val' theo điều kiện:** 
- nếu cột 'TITLE' có nội dung là 'Councilmember' thì 'random_val' sẽ có giá trị rand(), 
- nếu có nội dung là 'Mayor' thì 'random_val' sẽ có giá trị là 2,
ngược lại sẽ có giá trị là 0.

In [125]:
df_sub3 = df_sub3.withColumn('random_val', when(df.TITLE == 'Councilmember', rand())
                                .when(df.TITLE == 'Mayor', 2)
                                .otherwise(0)
)

#df_sub3 = df_sub3.drop('splits')
df_sub3.show(10)

+----------+--------------------+-------------------+--------------------+----------+---------+-------------------+
|      DATE|               TITLE|         VOTER_NAME|              splits|first_name|last_name|         random_val|
+----------+--------------------+-------------------+--------------------+----------+---------+-------------------+
|04/11/2018|Deputy Mayor Pro Tem|       Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|                0.0|
|02/14/2018|       Councilmember|    Lee M. Kleinman| [Lee, M., Kleinman]|       Lee| Kleinman| 0.5097770306735911|
|04/25/2018|       Councilmember|     Tennell Atkins|   [Tennell, Atkins]|   Tennell|   Atkins|0.06338329801520726|
|08/29/2018|       Councilmember|       Kevin Felder|     [Kevin, Felder]|     Kevin|   Felder|0.21978078770674647|
|10/18/2017|       Councilmember| Jennifer S.  Gates|[Jennifer, S., Ga...|  Jennifer|    Gates|0.37583989515812777|
|12/13/2017|       Councilmember|     Sandy  Greyson|    [Sandy, Greyson

### **12. Lọc các dòng dữ liệu có 'random_val' = 0. Hiển thị.**

In [126]:
df_sub4 = df_sub3.filter(df_sub3.random_val == 0)

In [127]:
df_sub4.show(5)

+----------+--------------------+-----------------+--------------------+----------+---------+----------+
|      DATE|               TITLE|       VOTER_NAME|              splits|first_name|last_name|random_val|
+----------+--------------------+-----------------+--------------------+----------+---------+----------+
|04/11/2018|Deputy Mayor Pro Tem|     Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|       0.0|
|04/12/2017|       Mayor Pro Tem| Monica R. Alonzo|[Monica, R., Alonzo]|    Monica|   Alonzo|       0.0|
|06/28/2017|Deputy Mayor Pro Tem|     Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|       0.0|
|01/03/2018|Deputy Mayor Pro Tem|     Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|       0.0|
|01/17/2018|       Mayor Pro Tem|Dwaine R. Caraway|[Dwaine, R., Cara...|    Dwaine|  Caraway|       0.0|
+----------+--------------------+-----------------+--------------------+----------+---------+----------+
only showing top 5 rows



### **13. Xây dựng function: getFirstAndMiddle(names) trả về kết quả gồm First và Middle (names).**
- Khai báo function vừa viết dưới dạng udf đặt tên là udfFirstAndMiddle

In [128]:
def getFirstAndMiddle(names):
    return ' '.join(names[:-1])

In [129]:
udfFirstAndMiddle = udf(getFirstAndMiddle, StringType())

### **14. Tạo cột first_and_middle_name bằng cách gọi udf trên với tham số truyền vào là cột 'splits'. In kết quả.**

In [130]:
df_sub4 = df_sub4.withColumn('first_and_middle_name', udfFirstAndMiddle(col('splits')))

In [131]:
df_sub4.show(5)

+----------+--------------------+-----------------+--------------------+----------+---------+----------+---------------------+
|      DATE|               TITLE|       VOTER_NAME|              splits|first_name|last_name|random_val|first_and_middle_name|
+----------+--------------------+-----------------+--------------------+----------+---------+----------+---------------------+
|04/11/2018|Deputy Mayor Pro Tem|     Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|       0.0|                 Adam|
|04/12/2017|       Mayor Pro Tem| Monica R. Alonzo|[Monica, R., Alonzo]|    Monica|   Alonzo|       0.0|            Monica R.|
|06/28/2017|Deputy Mayor Pro Tem|     Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|       0.0|                 Adam|
|01/03/2018|Deputy Mayor Pro Tem|     Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|       0.0|                 Adam|
|01/17/2018|       Mayor Pro Tem|Dwaine R. Caraway|[Dwaine, R., Cara...|    Dwaine|  Caraway|       0.0|       

### **15. Xóa bỏ các cột 'first_name', 'splits'. In kết quả.**

In [132]:
df_sub3 = df_sub3.drop(*['first_name', 'splits'])

In [133]:
df_sub3.show(5)

+----------+--------------------+------------------+---------+-------------------+
|      DATE|               TITLE|        VOTER_NAME|last_name|         random_val|
+----------+--------------------+------------------+---------+-------------------+
|04/11/2018|Deputy Mayor Pro Tem|      Adam Medrano|  Medrano|                0.0|
|02/14/2018|       Councilmember|   Lee M. Kleinman| Kleinman| 0.5097770306735911|
|04/25/2018|       Councilmember|    Tennell Atkins|   Atkins|0.06338329801520726|
|08/29/2018|       Councilmember|      Kevin Felder|   Felder|0.21978078770674647|
|10/18/2017|       Councilmember|Jennifer S.  Gates|    Gates|0.37583989515812777|
+----------+--------------------+------------------+---------+-------------------+
only showing top 5 rows



### **16. Thêm cột 'ROW_ID' bằng phương thức: monotonically_increasing_id() (trong pyspark.sql.functions)**

In [136]:
df_sub3 = df_sub3.select(df["VOTER_NAME"]).distinct()
print("\nThere are %d rows in the df DataFrame.\n" % df_sub3.count())


There are 27 rows in the df DataFrame.



In [137]:
df_sub3 = df_sub3.withColumn('ROW_ID', monotonically_increasing_id())

### **17. Hiển thị 10 dòng đầu của dữ liệu với ROW_ID tăng dần.**

In [139]:
df_sub3.orderBy(df_sub3.ROW_ID.desc()).show(10)

+-------------------+-------------+
|         VOTER_NAME|       ROW_ID|
+-------------------+-------------+
|       Lee Kleinman|1709396983808|
|        Erik Wilson|1700807049216|
|Carolyn King Arnold|1632087572480|
|Rickey D.  Callahan|1597727834112|
|   Monica R. Alonzo|1382979469312|
|    Lee M. Kleinman|1228360646656|
|  Jennifer S. Gates|1194000908288|
|Philip T.  Kingston|1185410973696|
|  Dwaine R. Caraway|1142461300736|
| Rickey D. Callahan|1125281431553|
+-------------------+-------------+
only showing top 10 rows



In [142]:
print("\nThere are %d partitions in the df DataFrame.\n" % df_sub3.rdd.getNumPartitions())


There are 200 partitions in the df DataFrame.



In [143]:
'''
=> 200 partitions
'''

'\n=> 200 partitions\n'

In [145]:
#Determine the highest ROW_ID and save it in previous_max_ID
previous_max_ID = df_sub3.select('ROW_ID').rdd.max()[0]

# Add a ROW_ID column to df_april starting at the desired value
voter_df_april = df_sub3.withColumn('ROW_ID', monotonically_increasing_id() + previous_max_ID)

In [146]:
# Show the ROW_ID from both DataFrames and compare
df_sub3.select('ROW_ID').show(5)
voter_df_april.select('ROW_ID').show(5)

+------------+
|      ROW_ID|
+------------+
|  8589934592|
| 34359738368|
| 42949672960|
| 51539607552|
|103079215104|
+------------+
only showing top 5 rows

+-------------+
|       ROW_ID|
+-------------+
|1717986918400|
|1743756722176|
|1752346656768|
|1760936591360|
|1812476198912|
+-------------+
only showing top 5 rows

